In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import numpy as np
import pandas as pd
import joblib

import os
from tqdm import tqdm
from glob import glob
import matplotlib.pyplot as plt


import pathlib
DATA_DIR = pathlib.Path.cwd()/'data/input'
OUT_DIR = pathlib.Path.cwd()/'data/output'

import sys 
sys.path.append(str(pathlib.Path.cwd()/'utils'))
from utils.misc_utils import fullrange, realized_volatility, log_return, rmspe, get_stock_path, load_parquet_file, load_parquet_files, load_train_test
from utils.feature_engineering_utils import full_feature_engineering, groupby_and_aggregate, generate_features_book_data, generate_features_trade_data, full_feature_engineering_by_cutoff

book_aggregation = {
    'wap1': [np.mean, np.std, fullrange], 
    'wap2': [np.mean, np.std, fullrange], 
    'log_return_1': [fullrange, np.sum, np.mean, realized_volatility], 
    'log_return_2': [fullrange, np.sum, np.mean, realized_volatility], 
    'bid_ask_price_spread_1': [np.mean, np.std, fullrange],
    'bid_ask_price_spread_2': [np.mean, np.std, fullrange],
    'bid_ask_size_spread_1': [np.mean, np.std, fullrange],
    'bid_ask_size_spread_2': [np.mean, np.std, fullrange]
    }

trade_aggregation = {
    'volume': [np.mean, np.sum, np.std], 
    'price': [np.mean, np.std], 
    'order_count': [np.mean, np.sum, np.std]
    }

time_agg_trade = {
    'volume_mean': [np.mean, np.sum, np.std], 
    'price_mean': [np.mean, np.std], 
    'order_count_mean': [np.mean, np.sum, np.std]
    }

time_agg_book = {
    'wap1_std': [np.mean], 
    'wap2_std': [np.mean], 
    'log_return_1_realized_volatility': [np.mean], 
    'log_return_2_realized_volatility': [np.mean], 
    'log_return_1_sum': [np.mean], 
    'log_return_2_sum': [np.mean] 
    }

stock_agg_trade = {
    'volume_mean': [np.mean,np.std], 
    'price_mean': [np.mean, np.std], 
    'order_count_mean': [np.mean, np.std]
    }

stock_agg_book = {
    'wap1_std': [np.std], 
    'wap2_std': [np.std], 
    'log_return_1_realized_volatility': [np.std], 
    'log_return_2_realized_volatility': [np.std], 
    'log_return_1_sum': [np.std], 
    'log_return_2_sum': [np.std] 
    }

pd.set_option('display.max_columns', None)

In [3]:
training_target = pd.read_csv(DATA_DIR/'train.csv')
book = load_parquet_files(stock_ids=[0, 126], file_type='book')
trade = load_parquet_files(stock_ids=[0, 126], file_type='trade')

100%|██████████| 2/2 [00:00<00:00,  6.37it/s]


In [4]:
book = generate_features_book_data(book)
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,id,wap1,wap2,log_return_1,log_return_2,bid_ask_price_spread_1,bid_ask_price_spread_2,bid_ask_size_spread_1,bid_ask_size_spread_2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0,0-5,1.001434,1.001390,0.000000,0.000000,0.000879,0.000983,223,98
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0,0-5,1.001448,1.001390,0.000014,0.000000,0.000879,0.000983,97,98
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0,0-5,1.001448,1.001391,0.000000,0.000001,0.000879,0.001034,97,98
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,0-5,1.001443,1.001391,-0.000005,0.000000,0.000879,0.001034,123,98
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,0-5,1.001443,1.001391,0.000000,0.000000,0.000879,0.001034,123,98


In [5]:
agg_book_data = groupby_and_aggregate(book, agg_col = 'id', agg_dict=book_aggregation)
agg_book_data.head()

,wap1_mean,wap1_std,wap1_fullrange,wap2_mean,wap2_std,wap2_fullrange,log_return_1_fullrange,log_return_1_sum,log_return_1_mean,log_return_1_realized_volatility,log_return_2_fullrange,log_return_2_sum,log_return_2_mean,log_return_2_realized_volatility,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange
id,,,,,,,,,,,,,,,,,,,,,,,,,,
0-1000,0.998800,0.000531,0.001829,0.998738,0.000538,0.002868,0.001286,-0.000944,-0.000006,0.001781,0.002234,-0.001198,-0.000007,0.004389,0.000443,0.000173,0.000878,0.000765,0.000249,0.001478,11.695122,171.482507,1116,28.609756,83.794630,599
0-10000,0.999884,0.000395,0.001409,0.999912,0.000488,0.002080,0.001733,0.000546,0.000002,0.002890,0.001641,0.000686,0.000002,0.004111,0.000460,0.000232,0.001022,0.000739,0.000239,0.001314,10.143357,152.601616,1045,-17.912587,130.656572,747
0-10005,1.001301,0.001809,0.007130,1.001160,0.001676,0.007296,0.004685,0.002516,0.000014,0.008674,0.006666,0.000928,0.000005,0.013725,0.002107,0.000602,0.003449,0.002743,0.000602,0.003025,-11.918478,103.390706,532,28.766304,107.369872,694
0-10017,0.996141,0.004763,0.017300,0.996030,0.004678,0.018040,0.016086,-0.002971,-0.000013,0.017629,0.018510,-0.003512,-0.000015,0.021224,0.003538,0.001323,0.007071,0.004959,0.002062,0.010697,60.934211,158.634259,860,-0.675439,151.850436,700
0-10030,0.999464,0.000433,0.001729,0.999477,0.000498,0.002206,0.001503,0.000580,0.000003,0.002551,0.004305,0.003158,0.000016,0.005463,0.000623,0.000246,0.001085,0.001040,0.000305,0.001463,33.577320,83.867072,364,6.876289,85.745528,416


In [6]:
trade = generate_features_trade_data(trade)
trade.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id,id,volume
0,5,21,1.002301,326,12,0,0-5,326.750244
1,5,46,1.002778,128,4,0,0-5,128.355591
2,5,50,1.002818,55,1,0,0-5,55.155014
3,5,57,1.003155,121,5,0,0-5,121.381798
4,5,68,1.003646,4,1,0,0-5,4.014584


In [7]:
agg_trade_data =  groupby_and_aggregate(trade, agg_col = 'id', agg_dict=trade_aggregation)
agg_trade_data.head()

,volume_mean,volume_sum,volume_std,price_mean,price_std,order_count_mean,order_count_sum,order_count_std
id,,,,,,,,
0-1000,29.961796,898.853882,45.738499,0.998908,0.000554,2.133333,64,2.029665
0-10000,80.825371,1939.808960,132.549789,0.999931,0.000373,2.916667,70,2.500725
0-10005,81.590820,2366.133789,119.623756,1.001232,0.001756,3.137931,91,3.502286
0-10017,89.547050,3760.976074,133.678314,0.996374,0.004358,3.166667,133,3.875606
0-10030,98.030350,2940.910645,103.179214,0.999314,0.000493,2.333333,70,1.667816


In [8]:
output_df = agg_book_data.merge(agg_trade_data, left_index=True, right_index=True).reset_index()
output_df['stock_id'] = output_df['id'].apply(lambda x: int(x.split('-')[0]))
output_df['time_id'] = output_df['id'].apply(lambda x: int(x.split('-')[1]))
output_df.head()

,id,wap1_mean,wap1_std,wap1_fullrange,wap2_mean,wap2_std,wap2_fullrange,log_return_1_fullrange,log_return_1_sum,log_return_1_mean,log_return_1_realized_volatility,log_return_2_fullrange,log_return_2_sum,log_return_2_mean,log_return_2_realized_volatility,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange,volume_mean,volume_sum,volume_std,price_mean,price_std,order_count_mean,order_count_sum,order_count_std,stock_id,time_id
0,0-1000,0.998800,0.000531,0.001829,0.998738,0.000538,0.002868,0.001286,-0.000944,-0.000006,0.001781,0.002234,-0.001198,-0.000007,0.004389,0.000443,0.000173,0.000878,0.000765,0.000249,0.001478,11.695122,171.482507,1116,28.609756,83.794630,599,29.961796,898.853882,45.738499,0.998908,0.000554,2.133333,64,2.029665,0,1000
1,0-10000,0.999884,0.000395,0.001409,0.999912,0.000488,0.002080,0.001733,0.000546,0.000002,0.002890,0.001641,0.000686,0.000002,0.004111,0.000460,0.000232,0.001022,0.000739,0.000239,0.001314,10.143357,152.601616,1045,-17.912587,130.656572,747,80.825371,1939.808960,132.549789,0.999931,0.000373,2.916667,70,2.500725,0,10000
2,0-10005,1.001301,0.001809,0.007130,1.001160,0.001676,0.007296,0.004685,0.002516,0.000014,0.008674,0.006666,0.000928,0.000005,0.013725,0.002107,0.000602,0.003449,0.002743,0.000602,0.003025,-11.918478,103.390706,532,28.766304,107.369872,694,81.590820,2366.133789,119.623756,1.001232,0.001756,3.137931,91,3.502286,0,10005
3,0-10017,0.996141,0.004763,0.017300,0.996030,0.004678,0.018040,0.016086,-0.002971,-0.000013,0.017629,0.018510,-0.003512,-0.000015,0.021224,0.003538,0.001323,0.007071,0.004959,0.002062,0.010697,60.934211,158.634259,860,-0.675439,151.850436,700,89.547050,3760.976074,133.678314,0.996374,0.004358,3.166667,133,3.875606,0,10017
4,0-10030,0.999464,0.000433,0.001729,0.999477,0.000498,0.002206,0.001503,0.000580,0.000003,0.002551,0.004305,0.003158,0.000016,0.005463,0.000623,0.000246,0.001085,0.001040,0.000305,0.001463,33.577320,83.867072,364,6.876289,85.745528,416,98.030350,2940.910645,103.179214,0.999314,0.000493,2.333333,70,1.667816,0,10030


## Time Period Aggregation

In [9]:
time_agg_trade_data = groupby_and_aggregate(output_df, agg_col = 'time_id', agg_dict=time_agg_trade, suffix='_period')
time_agg_trade_data.head()

,volume_mean_mean_period,volume_mean_sum_period,volume_mean_std_period,price_mean_mean_period,price_mean_std_period,order_count_mean_mean_period,order_count_mean_sum_period,order_count_mean_std_period
time_id,,,,,,,,
5,63.489216,126.978432,23.001141,1.002913,0.001144,2.669118,5.338235,0.114385
11,75.123787,150.247574,45.460377,1.000473,0.000377,2.709259,5.418519,1.144465
16,109.012894,218.025787,32.058147,0.999548,0.000486,2.744615,5.489231,0.034811
31,81.574921,163.149841,69.389015,0.998861,0.000225,3.050000,6.100000,1.249222
62,67.259079,134.518158,19.963213,0.999277,0.000483,3.320025,6.640049,1.025913


In [10]:
time_agg_book_data = groupby_and_aggregate(output_df, agg_col = 'time_id', agg_dict=time_agg_book, suffix='_period')
time_agg_book_data.head()

,wap1_std_mean_period,wap2_std_mean_period,log_return_1_realized_volatility_mean_period,log_return_2_realized_volatility_mean_period,log_return_1_sum_mean_period,log_return_2_sum_mean_period
time_id,,,,,,
5,0.000876,0.000990,0.006064,0.009029,0.002922,0.002750
11,0.000503,0.000533,0.003823,0.004540,-0.002060,-0.001627
16,0.000802,0.000840,0.002699,0.005010,-0.002588,-0.002912
31,0.000809,0.000708,0.004144,0.004617,-0.000918,-0.000466
62,0.000413,0.000478,0.003319,0.004168,0.001852,0.001475


In [11]:
time_agg = time_agg_book_data.merge(time_agg_trade_data, left_index=True, right_index=True)
output_df_with_time = output_df.merge(time_agg, on='time_id')
output_df_with_time = output_df_with_time.merge(training_target, on=['stock_id', 'time_id']).head()

In [12]:
train, test = load_train_test()
train_id = train.stock_id.unique()
test_id = test.stock_id.unique()

In [13]:
#final_training_data = full_feature_engineering(stock_ids = [], train=True, training_target= training_target)
#final_test_data = full_feature_engineering(stock_ids = [], train=False)

cutoffs =  [(0,150), (150, 300), (300, 450), (450, 600)]
cutoffs =  [(0,100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)]

final_training_data = full_feature_engineering_by_cutoff(
    cutoffs = cutoffs,
    stock_ids = train_id,
    training=True)

final_test_data = full_feature_engineering_by_cutoff(
    cutoffs = cutoffs,
    stock_ids = test_id,
    training=False)

#final_training_data = train.merge(final_training_data, on = ['id', 'stock_id', 'time_id'], how='left')
#final_test_data = test.merge(final_test_data, on = ['id', 'stock_id', 'time_id'], how='left')

final_training_data.to_pickle(OUT_DIR/'final_training_data_finer_buckets.pkl')
final_test_data.to_pickle(OUT_DIR/'final_test_data_finer_buckets.pkl')


100%|██████████| 1/1 [00:00<00:00, 21.74it/s]


In [15]:
final_training_data.head()

,id,time_id,stock_id,target,wap1_mean_0_100,wap1_std_0_100,wap1_fullrange_0_100,wap2_mean_0_100,wap2_std_0_100,wap2_fullrange_0_100,log_return_1_fullrange_0_100,log_return_1_sum_0_100,log_return_1_mean_0_100,log_return_1_realized_volatility_0_100,log_return_2_fullrange_0_100,log_return_2_sum_0_100,log_return_2_mean_0_100,log_return_2_realized_volatility_0_100,bid_ask_price_spread_1_mean_0_100,bid_ask_price_spread_1_std_0_100,bid_ask_price_spread_1_fullrange_0_100,bid_ask_price_spread_2_mean_0_100,bid_ask_price_spread_2_std_0_100,bid_ask_price_spread_2_fullrange_0_100,bid_ask_size_spread_1_mean_0_100,bid_ask_size_spread_1_std_0_100,bid_ask_size_spread_1_fullrange_0_100,bid_ask_size_spread_2_mean_0_100,bid_ask_size_spread_2_std_0_100,bid_ask_size_spread_2_fullrange_0_100,volume_mean_0_100,volume_sum_0_100,volume_std_0_100,price_mean_0_100,price_std_0_100,order_count_mean_0_100,order_count_sum_0_100,order_count_std_0_100,wap1_std_mean_period_0_100,wap2_std_mean_period_0_100,log_return_1_realized_volatility_mean_period_0_100,log_return_2_realized_volatility_mean_period_0_100,log_return_1_sum_mean_period_0_100,log_return_2_sum_mean_period_0_100,volume_mean_mean_period_0_100,volume_mean_sum_period_0_100,volume_mean_std_period_0_100,price_mean_mean_period_0_100,price_mean_std_period_0_100,order_count_mean_mean_period_0_100,order_count_mean_sum_period_0_100,order_count_mean_std_period_0_100,wap1_std_std_stock_0_100,wap2_std_std_stock_0_100,log_return_1_realized_volatility_std_stock_0_100,log_return_2_realized_volatility_std_stock_0_100,log_return_1_sum_std_stock_0_100,log_return_2_sum_std_stock_0_100,volume_mean_mean_stock_0_100,volume_mean_std_stock_0_100,price_mean_mean_stock_0_100,price_mean_std_stock_0_100,order_count_mean_mean_stock_0_100,order_count_mean_std_stock_0_100,wap1_mean_100_200,wap1_std_100_200,wap1_fullrange_100_200,wap2_mean_100_200,wap2_std_100_200,wap2_fullrange_100_200,log_return_1_fullrange_100_200,log_return_1_sum_100_200,log_return_1_mean_100_200,log_return_1_realized_volatility_100_200,log_return_2_fullrange_100_200,log_return_2_sum_100_200,log_return_2_mean_100_200,log_return_2_realized_volatility_100_200,bid_ask_price_spread_1_mean_100_200,bid_ask_price_spread_1_std_100_200,bid_ask_price_spread_1_fullrange_100_200,bid_ask_price_spread_2_mean_100_200,bid_ask_price_spread_2_std_100_200,bid_ask_price_spread_2_fullrange_100_200,bid_ask_size_spread_1_mean_100_200,bid_ask_size_spread_1_std_100_200,bid_ask_size_spread_1_fullrange_100_200,bid_ask_size_spread_2_mean_100_200,bid_ask_size_spread_2_std_100_200,bid_ask_size_spread_2_fullrange_100_200,volume_mean_100_200,volume_sum_100_200,volume_std_100_200,price_mean_100_200,price_std_100_200,order_count_mean_100_200,order_count_sum_100_200,order_count_std_100_200,wap1_std_mean_period_100_200,wap2_std_mean_period_100_200,log_return_1_realized_volatility_mean_period_100_200,log_return_2_realized_volatility_mean_period_100_200,log_return_1_sum_mean_period_100_200,log_return_2_sum_mean_period_100_200,volume_mean_mean_period_100_200,volume_mean_sum_period_100_200,volume_mean_std_period_100_200,price_mean_mean_period_100_200,price_mean_std_period_100_200,order_count_mean_mean_period_100_200,order_count_mean_sum_period_100_200,order_count_mean_std_period_100_200,wap1_std_std_stock_100_200,wap2_std_std_stock_100_200,log_return_1_realized_volatility_std_stock_100_200,log_return_2_realized_volatility_std_stock_100_200,log_return_1_sum_std_stock_100_200,log_return_2_sum_std_stock_100_200,volume_mean_mean_stock_100_200,volume_mean_std_stock_100_200,price_mean_mean_stock_100_200,price_mean_std_stock_100_200,order_count_mean_mean_stock_100_200,order_count_mean_std_stock_100_200,wap1_mean_200_300,wap1_std_200_300,wap1_fullrange_200_300,wap2_mean_200_300,wap2_std_200_300,wap2_fullrange_200_300,log_return_1_fullrange_200_300,log_return_1_sum_200_300,log_return_1_mean_200_300,log_return_1_realized_volatility_200_300,log_return_2_fullrange_200_300,log_return_2_sum_200_300,log_return_2_me

In [16]:
final_training_data.isna().sum().sort_values()

id                                    0
price_std_0_100                       0
order_count_mean_0_100                0
order_count_sum_0_100                 0
order_count_std_0_100                 0
                                     ..
wap1_std_400_500                      4
wap1_fullrange_400_500                4
wap2_mean_400_500                     4
wap2_fullrange_400_500                4
order_count_mean_std_stock_500_600    4
Length: 364, dtype: int64

## Testing Joblib

In [1]:
from utils.feature_engineering_utils import full_feature_engineering

In [2]:
import numpy as np
import pandas as pd
import joblib

import os
from tqdm import tqdm
import pathlib
from utils.misc_utils import fullrange, realized_volatility, log_return, get_stock_path, load_parquet_file, load_parquet_files, load_train_test
from utils.feature_engineering_utils import generate_features_book_data, generate_features_trade_data, groupby_and_aggregate

training_target, test = load_train_test()
id_list = list(training_target.stock_id.unique())[:24]

competition = 'optiver-realized-volatility-prediction'
DATA_DIR = pathlib.Path.cwd()/'data/input'
OUT_DIR = pathlib.Path.cwd()/'data/output'

book_aggregation = {
    'wap1': [np.mean, np.std, fullrange], 
    'wap2': [np.mean, np.std, fullrange], 
    'log_return_1': [fullrange, np.sum, np.mean, realized_volatility], 
    'log_return_2': [fullrange, np.sum, np.mean, realized_volatility], 
    'bid_ask_price_spread_1': [np.mean, np.std, fullrange],
    'bid_ask_price_spread_2': [np.mean, np.std, fullrange],
    'bid_ask_size_spread_1': [np.mean, np.std, fullrange],
    'bid_ask_size_spread_2': [np.mean, np.std, fullrange]
    }

trade_aggregation = {
    'volume': [np.mean, np.sum, np.std], 
    'price': [np.mean, np.std], 
    'order_count': [np.mean, np.sum, np.std]
    }

time_agg_trade = {
    'volume_mean': [np.mean, np.sum, np.std], 
    'price_mean': [np.mean, np.std], 
    'order_count_mean': [np.mean, np.sum, np.std]
    }

time_agg_book = {
    'wap1_std': [np.mean], 
    'wap2_std': [np.mean], 
    'log_return_1_realized_volatility': [np.mean], 
    'log_return_2_realized_volatility': [np.mean], 
    'log_return_1_sum': [np.mean], 
    'log_return_2_sum': [np.mean] 
    }

stock_agg_trade = {
    'volume_mean': [np.mean,np.std], 
    'price_mean': [np.mean, np.std], 
    'order_count_mean': [np.mean, np.std]
    }

stock_agg_book = {
    'wap1_std': [np.std], 
    'wap2_std': [np.std], 
    'log_return_1_realized_volatility': [np.std], 
    'log_return_2_realized_volatility': [np.std], 
    'log_return_1_sum': [np.std], 
    'log_return_2_sum': [np.std] 
    }

def full_feature_engineering_no_joblib(
    stock_ids: list = [],
    training_target: pd.DataFrame = training_target, 
    book_aggregation: dict = book_aggregation,
    trade_aggregation: dict = trade_aggregation,
    time_agg_book: dict = time_agg_book,
    time_agg_trade: dict = time_agg_trade,
    training: bool =True, 
    lower_seconds_cutoff: int = 0,
    upper_seconds_cutoff: int = 600
    ): 
    processed_stock_data_list = []
    
    for stock_id in stock_ids:
        if training:
            book = load_parquet_file(str(DATA_DIR)+ f'/book_train.parquet/stock_id={stock_id}')
            trade = load_parquet_file(str(DATA_DIR)+ f'/trade_train.parquet/stock_id={stock_id}')

        else: 
            book = load_parquet_file(str(DATA_DIR)+ f'/book_test.parquet/stock_id={stock_id}')     
            trade = load_parquet_file(str(DATA_DIR)+ f'/trade_test.parquet/stock_id={stock_id}')     

        # Important to generate the features first in order to have return for the first non-zero time cutoff
        book = generate_features_book_data(book)
        trade = generate_features_trade_data(trade)
                                             
        book = book.loc[(book.seconds_in_bucket>=lower_seconds_cutoff) & (book.seconds_in_bucket<upper_seconds_cutoff)]
        trade = trade.loc[(trade.seconds_in_bucket>=lower_seconds_cutoff) & (trade.seconds_in_bucket<upper_seconds_cutoff)]
        
        agg_book_data = groupby_and_aggregate(book, agg_col = 'id', agg_dict=book_aggregation)
        agg_trade_data =  groupby_and_aggregate(trade, agg_col = 'id', agg_dict=trade_aggregation)
        
        # Fillna with 0 make sense because in the data where we don't have trade data that means no trading had occured
        merged_df = agg_book_data.merge(agg_trade_data, left_index=True, right_index=True, how='left').fillna(0).reset_index()
        merged_df['stock_id'] = merged_df['id'].apply(lambda x: int(x.split('-')[0]))
        merged_df['time_id'] = merged_df['id'].apply(lambda x: int(x.split('-')[1]))
        processed_stock_data_list.append(merged_df)
        
    processed_stock_data = pd.concat(processed_stock_data_list)

    time_agg_trade_data = groupby_and_aggregate(processed_stock_data, agg_col = 'time_id', agg_dict=time_agg_trade, suffix='_period')
    time_agg_book_data = groupby_and_aggregate(processed_stock_data, agg_col = 'time_id', agg_dict=time_agg_book, suffix='_period')
    time_agg = time_agg_book_data.merge(time_agg_trade_data, left_index=True, right_index=True).reset_index()

    stock_agg_trade_data = groupby_and_aggregate(processed_stock_data, agg_col = 'stock_id', agg_dict=stock_agg_trade, suffix='_stock')
    stock_agg_book_data = groupby_and_aggregate(processed_stock_data, agg_col = 'stock_id', agg_dict=stock_agg_book, suffix='_stock')
    stock_agg = stock_agg_book_data.merge(stock_agg_trade_data, left_index=True, right_index=True).reset_index()

    processed_stock_data = processed_stock_data.merge(time_agg, on='time_id', how='left').merge(stock_agg, on='stock_id', how='left')
        
    if training:
        processed_stock_data =  training_target.merge(processed_stock_data, on=['id', 'stock_id', 'time_id'], how='left')

    return processed_stock_data


In [15]:
data1 = full_feature_engineering_no_joblib(stock_ids=id_list)

100%|██████████| 24/24 [00:21<00:00,  1.11it/s]


In [16]:
data2 = full_feature_engineering(stock_ids=id_list)

100%|██████████| 24/24 [02:22<00:00,  5.93s/it]


## Creating One Train Pipeline and Model Registry

In [1]:
from train import optiver_train_and_log_experiment

cutoffs =  [(0,150), (150, 300), (300, 450), (450, 600)]
run_name = f'LOFO_new_features_{cutoffs}'

cutoffs_list =  [
    [(0,600)], 
    [(0,300), (300, 600)], 
    [(0,200), (200, 400), (400, 600)],  
    [(0,150), (150, 300), (300, 450), (450, 600)],
    [(0,100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)],
    [(0,600), (150, 600), (300, 600), (450, 600)]
]

for cutoffs in cutoffs_list:
    run_name = f'LOFO_{cutoffs}'
    optiver_train_and_log_experiment(
        run_name=run_name, 
        cutoffs=cutoffs, 
        lofo_threshold = 0 
    )

2021-09-17 08:02:31,414 - logfile - INFO - ...Currently Running LOFO_[(0, 600)]...
2021-09-17 08:02:32,488 - logfile - INFO - ...Currently Feature Engineering...
 50%|█████     | 56/112 [02:12<02:27,  2.64s/it]C:\Users\twongsan\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 112/112 [04:37<00:00,  2.48s/it]
2021-09-17 08:07:48,342 - logfile - INFO - ...Currently Training First LGBM...


[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.089903 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Using self-defined objective function
[1]	valid_0's rmspe: 0.908043
Training until validation scores don't improve for 200 rounds
[2]	valid_0's rmspe: 0.825933
[3]	valid_0's rmspe: 0.752655
[4]	valid_0's rmspe: 0.687506
[5]	valid_0's rmspe: 0.629602
[6]	valid_0's rmspe: 0.578295
[7]	valid_0's rmspe: 0.533049
[8]	valid_0's rmspe: 0.493149
[9]	valid_0's rmspe: 0.457993
[10]	valid_0's rmspe: 0.427258
[11]	valid_0's rmspe: 0.400503
[12]	valid_0's rmspe: 0.37725
[13]	valid_0's rmspe: 0.356986
[14]	valid_0's rmspe: 0.339518
[15]	valid_0's rmspe: 0.324387
[16]	valid_0's rmspe: 0.311511
[17]	valid_0's rmspe: 0.300593
[18]	valid_0's rmspe: 0.29128
[19]	valid_0's rmspe: 0.283268
[20]	valid_0's rmspe: 0.276425
[21]	valid_0's rmspe: 0.270726
[22]	valid_0's 

2021-09-17 08:09:01,343 - logfile - INFO - ...Currently Performing LOFO...
100%|██████████| 63/63 [00:22<00:00,  2.78it/s]


MlflowException: Changing param values is not allowed. Param with key='Feature Importance DF' was already logged with value='0' for run ID='4aac7cf6e9344e3abc055a3c2374e501'. Attempted logging new value '{18: {'feature': 'log_return_gap_sum_0_600', 'importance_mean': 0.06534379811199129, 'importance_std': 0.04316617885292415, 'val_imp_0': 0.036993441204729305, 'val_imp_1': 0.13430446260389395, 'val_imp_2': 0.021915479937853755, 'val_imp_3': 0.06816180870148814}, 35: {'feature': 'wap_gap_std_mean_period_0_600', 'importance_mean': 0.04021951173200135, 'importance_std': 0.04709548803022122, 'val_imp_0': -0.00695516982373362, 'val_imp_1': 0.10982129359009585, 'val_imp_2': 0.0011618065431299174, 'val_imp_3': 0.05685011661851325}, 58: {'feature': 'volume_mean_std_stock_0_600', 'importance_mean': 0.02310011385199813, 'importance_std': 0.03376685680815597, 'val_imp_0': -0.005848890506577831, 'val_imp_1': 0.08063152336012969, 'val_imp_2': 0.010295114392239402, 'val_imp_3': 0.0073227081622012635